In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys; 
#sys.path.append('/home/mitch/school/data/mexico_enigh/src/')
sys.path.append('../src')
sys.path.append('/home/mitch/school/data/mexico_enigh/dicts/')
sys.path.append('/home/mitch/util/python/')
import json_utils

import utils
import os

from simpledbf import Dbf5

raw  = '/home/mitch/Dropbox/data/mexico_enigh/raw/'
interim = '/home/mitch/Dropbox/data/mexico_enigh/interim2/'
dicts = '/home/mitch/school/data/mexico_enigh/dicts/'

SQLalchemy is not installed. No support for SQL output.


In [2]:
os.chdir(dicts)
expend_cat = json_utils.load_json("capital_expenditures.json")
rename_capital_expenditures = json_utils.load_json("rename_capital.json")

years = ['1994', '1996']

dfs_out = []

In [3]:
expend_cat

{'Q001': 'expense_deposits_savings',
 'Q002': 'expense_loans_to_third_parties',
 'Q003': 'expense_credit_card_payments',
 'Q004': 'expense_debt_to_work_or_person',
 'Q005': 'expense_foreigncurrencies_perciousmetals_art',
 'Q006': 'expense_life_insurance',
 'Q007': 'expense_inheritance_dowrie_legacies',
 'Q008': 'expense_realestate_hh_not_inhabit',
 'Q009': 'expense_realestate_hh_inhabit',
 'Q010': 'expense_realestate_mortgage',
 'Q011': 'expense_other',
 'Q012': 'expense_hhbusiness_machinery_animal_equipment',
 'Q013': 'expense_hhbusiness_negative_balance',
 'Q014': 'expense_securities_shares_bond',
 'Q015': 'expense_trademarks_patents_copyrights'}

In [4]:
year = '1996'
os.chdir(raw + year)

df = Dbf5('capital_expenditures.dbf').to_dataframe()
df.columns = [x.lower() for x in df.columns]
df = df.rename(columns = rename_capital_expenditures)
df = df.sort_values(['hhid', 'category'])
df.head(20)

,hhid,meses,category,expenses,ero1,ero2,ero3,ero4,ero5,ero6
0,19960110030,070605040302,Q001,1800.0,0.0,500.0,600.0,700.0,800.0,1000.0
1,19960110040,070605040302,Q001,750.0,0.0,1500.0,0.0,0.0,0.0,0.0
2,19960110070,070605040302,Q006,1800.0,3600.0,0.0,0.0,0.0,0.0,0.0
3,19960110090,070605040302,Q001,950.0,300.0,300.0,300.0,500.0,300.0,200.0
4,19960110110,070605040302,Q001,2100.0,2000.0,500.0,1000.0,700.0,0.0,0.0
5,19960110110,070605040302,Q004,1500.0,500.0,500.0,500.0,500.0,500.0,500.0
6,19960110120,070605040302,Q001,6000.0,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0
7,19960110140,070605040302,Q001,450.0,150.0,150.0,150.0,150.0,150.0,150.0
8,19960110150,070605040302,Q001,3900.0,4300.0,300.0,300.0,300.0,2300.0,300.0
9,19960110150,070605040302,Q003,4700.0,1650.0,1650.0,1650.0,1650.0,1650.0,1150.0


In [5]:
dfs = []
dfs_has = []
for year in years:
    os.chdir(raw + year)
    df = Dbf5('capital_expenditures.dbf').to_dataframe()
    df.columns = [x.lower() for x in df.columns]
    df = df.rename(columns = rename_capital_expenditures)
    df = df.sort_values(['hhid', 'category'])
    hhid     = np.sort(df['hhid'].unique()).astype('int')

    df['hhid'] = df['hhid'].astype(str)
    df = df[df['hhid'].apply(lambda x : x.isnumeric())]

    df['category'] = df['category'].astype('str')
    df = df[df['category'].apply(lambda x : x in expend_cat.keys())]
    df['category'] = df['category'].apply(lambda x : expend_cat[x])

    category = expend_cat.values()

    # sum over individual expenditures within each category
    df = df.groupby(['hhid', 'category'])['expenses'].apply(np.sum).reset_index()

    df = df.sort_values(['hhid', 'category'])
    df['has'] = (df['expenses'] > 0) + 0
    df_has = df[['hhid', 'category', 'has']].pivot(index='hhid', columns='category', values='has').reset_index()

    df_has = df_has.astype(np.float)
    df_has[np.isnan(df_has)] = 0
    df_has['hhid'] = df_has['hhid'].astype(int)

    df_has.columns = ['has_' + x  if x != 'hhid' else x for x in df_has.columns]

    df = df.drop(columns='has')
    df = df.pivot(index='hhid', columns='category', values='expenses').reset_index()
    df['hhid'] = df['hhid'].astype(int)
    df[np.isnan(df)] = 0.0

    dfs.append(df)
    dfs_has.append(df_has)


<ipython-input-5-e065c892e3b8>:27: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_has = df_has.astype(np.float)
<ipython-input-5-e065c892e3b8>:27: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_has = df_has.astype(np.float)


In [6]:
df = pd.concat(dfs)
df_has = pd.concat(dfs_has)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11367 entries, 0 to 5920
Data columns (total 16 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   hhid                                           11367 non-null  int64  
 1   expense_credit_card_payments                   11367 non-null  float64
 2   expense_debt_to_work_or_person                 11367 non-null  float64
 3   expense_deposits_savings                       11367 non-null  float64
 4   expense_foreigncurrencies_perciousmetals_art   11367 non-null  float64
 5   expense_hhbusiness_machinery_animal_equipment  11367 non-null  float64
 6   expense_hhbusiness_negative_balance            11367 non-null  float64
 7   expense_life_insurance                         11367 non-null  float64
 8   expense_loans_to_third_parties                 11367 non-null  float64
 9   expense_other                                  1136

In [8]:
df_has.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11367 entries, 0 to 5920
Data columns (total 16 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   hhid                                               11367 non-null  int64  
 1   has_expense_credit_card_payments                   11367 non-null  float64
 2   has_expense_debt_to_work_or_person                 11367 non-null  float64
 3   has_expense_deposits_savings                       11367 non-null  float64
 4   has_expense_foreigncurrencies_perciousmetals_art   11367 non-null  float64
 5   has_expense_hhbusiness_machinery_animal_equipment  11367 non-null  float64
 6   has_expense_hhbusiness_negative_balance            11367 non-null  float64
 7   has_expense_life_insurance                         11367 non-null  float64
 8   has_expense_loans_to_third_parties                 11367 non-null  float64
 9   has_exp

In [9]:
df = df.merge(df_has, on='hhid')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11367 entries, 0 to 11366
Data columns (total 31 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   hhid                                               11367 non-null  int64  
 1   expense_credit_card_payments                       11367 non-null  float64
 2   expense_debt_to_work_or_person                     11367 non-null  float64
 3   expense_deposits_savings                           11367 non-null  float64
 4   expense_foreigncurrencies_perciousmetals_art       11367 non-null  float64
 5   expense_hhbusiness_machinery_animal_equipment      11367 non-null  float64
 6   expense_hhbusiness_negative_balance                11367 non-null  float64
 7   expense_life_insurance                             11367 non-null  float64
 8   expense_loans_to_third_parties                     11367 non-null  float64
 9   expens

In [11]:
os.chdir(interim)
df.to_csv('capital_expenditures.csv', index=False)